# ElasticVectorSearch Integrations

## Install Packages

In [ ]:
#!pip3 install langchain

## Import packages

In [ ]:
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import ElasticVectorSearch
from langchain.text_splitter import CharacterTextSplitter
from urllib.request import urlopen
import os, json

load_dotenv()

openai_api_key=os.getenv('OPENAI_API_KEY')
elastic_cloud_id=os.getenv('ES_CLOUD_ID')
elastic_user=os.getenv('ES_USER')
elastic_password=os.getenv('ES_PASSWORD')
elastic_endpoint=os.getenv('ES_ENDPOINT')
elastic_index_name='byte-discuss-elastic-vector-search'


## Add documents

### Let's download the sample dataset and deserialize the document.

In [ ]:
url = "https://raw.githubusercontent.com/ashishtiwari1993/langchain-elasticsearch-RAG/main/data.json"

response = urlopen(url)

workplace_docs = json.loads(response.read())

### Split Documents into Passages

In [ ]:
metadata = []
content = []

for doc in workplace_docs:
  content.append(doc["content"])
  metadata.append({
      "name": doc["name"],
      "summary": doc["summary"],
      "rolePermissions":doc["rolePermissions"]
  })

text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
docs = text_splitter.create_documents(content, metadatas=metadata)

### Index data into elasticsearch

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

url = f"https://{elastic_user}:{elastic_password}@{elastic_endpoint}:9243"

es = ElasticVectorSearch.from_documents(
    docs,
    elasticsearch_url=url,
    index_name=elastic_index_name,
    embedding=embeddings
)

es

## Show Result

In [ ]:
def showResults(output):
  print("Total results: ", len(output))
  for index in range(len(output)):
    print(output[index])

## Similarity / Vector Search (KNN Search)

In [ ]:
query = "work from home policy"
result = es.similarity_search(query=query)

showResults(result)